# Final Exercise overview
The final exercise will comprise a bunch of mini-projects, together which will approximate a set of analysis tasks required during an emergency response.

The tasks will be categorized into different response functions, loosely according to the Incident Command Structure. We will initially put people into groups based on their interests, just to make sure every task has some people working on it, but people are free to move between tasks as needed. Be sure to communicate amongst the tasks to let each other know when you could use more people or have downtime and can offer help to others.

### Grid
First, we will go over the game grid.

In [1]:
# install and import dependencies
!pip install dwave_networkx
!pip install vrpy
import geopandas as gpd
import numpy as np
import dimod
import networkx as nx
import pandas as pd
import dwave_networkx as dnx
import seaborn as sns
import pickle
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from tqdm import tqdm
from vrpy import VehicleRoutingProblem

In [2]:
!wget 

wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1mX-mwgBz04nf8SOxKF2YVC3rkdbMa8PX' -O out.geojson

--2020-07-30 15:29:58--  https://docs.google.com/uc?export=download&id=1mX-mwgBz04nf8SOxKF2YVC3rkdbMa8PX
Resolving docs.google.com (docs.google.com)... 172.217.9.46, 2607:f8b0:4009:815::200e
Connecting to docs.google.com (docs.google.com)|172.217.9.46|:443... connected.
HTTP request sent, awaiting response... 

In [ ]:
# read game_grid as GeoDataFrame and correct CRS (Coordinate Reference System)
game_grid = gpd.read_file('out.geojson')
game_grid.crs='epsg:3857'
game_grid

The game grid uses the [Military Grid Reference System (MGRS)](https://en.wikipedia.org/wiki/Military_Grid_Reference_System) which breaks the world up into 1-km square cells. The MGRS column is an alphanumeric code which uniquely refers to each of those cells.

Each cell has some important attributes:
- Population: how many people live in that cell
- Health: Average health (higher better)
- Utility: Average access to utilities (higher better)
- Transport: How easy is it to move through the cell (higher better)
- SVI: Social Vulnerability Index
 

In [ ]:
game_grid.plot('SVI')

In [ ]:
game_grid.plot('Transport')

In [ ]:
game_grid.plot('Utility')

In [ ]:
game_grid.plot('Health')

### Facilities
There are also a number of facilities shapefiles in the `facilities` folders which will be used for a number of the mini-projects.
- airfields: places where airplanes and helicopters can be flown in/from
- cell_towers: communications infrastructure
- EMS: ambulance services
- fire_stations: fire services
- hospitals: hospitals
- local_eocs: local emergency operations centers
- powerplants: energy generating facilities
- locations: places where people can be tended to
- state_eocs: state emergency operatins centers

# Logistics
## Routing and Resource Management
### Asset deployment
You have access to 1 helicopter, 20 tractor-trailer trucks, and 20 pickup trucks. You can set up a base at any of the airfields of your choice. You can have up to 2 bases. Where do you place your assets?
- helicopter: cost \$1000/hr to operate, 250 km/hr speed - air, capacity: 2
- tractor-trailer: cost \$50/hr to operate, 100 km/hr speed - land, capacity: 20
- pickup truck: cost \$20/hr to operate, 120 km/hr speed - land, capacity: 3

### Game grid to network conversion
Convert the game grid gdf to a `networkx` object where each cell of the grid is a node, and it is connected to each of its adjacent nodes by an edge. The edge weight between node `node_i` and its neighbor `node_j` is given by the pseudocode formula:
`weight_ij = (5/2)*((1/node_i.transport) + (1/node_j.transport))`

where `node_i.transport` is the transport score of the cell `i`. The travel time across edge `ij` is given by `weight_ij*(120/speed_in_kmh)` minutes.

The helicopter flies straight line to its targets and ignores the network weights.

### Vehicle Routing Problem
You will be given lists of facilities (from the ones in `facilities/`) you need to deliver supplies to. Create a tool to compute the routes you would take and the vehicles you would use in order to get the supplies to those facilities. The vehicles must begin and end their trip at one of your bases, but they do not necessarily need to end at the same one they began at. You will be given a budget and time frame, and you want to deliver as many supplies as possible to those facilities within your time frame and budget. 

In [ ]:
game_grid # print out initial game_grid to check data

In [ ]:
# calculate basic flood risk heuristic based on distance to two coastline points
game_grid['FLOOD_RISK']=-np.minimum(np.sqrt((game_grid['x_centroid']+70.3)**2+(game_grid['y_centroid']-42.2)**2),
                                    np.sqrt((game_grid['x_centroid']+70.2)**2+(game_grid['y_centroid']-41.4)**2))
game_grid['FLOOD_RISK']-=np.min(game_grid['FLOOD_RISK'])
game_grid['FLOOD_RISK']/=np.max(game_grid['FLOOD_RISK'])

In [ ]:
# calculate risk based upon flood risk and SVI (Social Vulnerability Index)
game_grid['RISK']=game_grid['SVI']*(game_grid['FLOOD_RISK'])*0.9+0.1

In [ ]:
game_grid.plot('RISK')

In [ ]:
game_grid = game_grid.to_crs('epsg:4269') # convert geometry to latitude, longitude to match centroid location
game_grid['EAST'] = pd.to_numeric(game_grid['EASTING'].str[:-5]) # calculate grid east position from easting and ignore trailing 000mE
game_grid['NORTH'] = pd.to_numeric(game_grid['NORTHING'].str[:-5]) # calculate grid north position from northing and ignore trailing 000mN
game_grid_pos = game_grid.set_index(['EAST', 'NORTH']) # set index to grid position for seperate dataframe
game_grid = game_grid.set_index('MGRS') # use MGRS as main indexing method

In [ ]:
# create a graph from GeoDataFrame connected as a grid
G = nx.Graph() # create empty networkx graph
for pos, cell in game_grid_pos.iterrows():
    for neighbor_pos in [(pos[0]+1, pos[1]), # east
                         (pos[0]-1, pos[1]), # west
                         (pos[0],   pos[1]+1), # north
                         (pos[0],   pos[1]-1)]: # south
        if neighbor_pos in game_grid_pos.index:
            neighbor = game_grid_pos.loc[neighbor_pos] # find neighbor in this direction
            G.add_edge(cell['MGRS'], # cell MGRS id, used as node id
                       neighbor['MGRS'], # neightbor MGRS id, used as node id
                       weight = (1/cell['Transport'] + 1/neighbor['Transport']) * 5/2) # weight from definition

In [ ]:
nx.set_node_attributes(G, game_grid.to_dict('index')) # use pandas DataFrame for node data

In [ ]:
bases = ('19TCG1685','19TCG1752') # choose two bases for routes

In [ ]:
positions=game_grid[['x_centroid', 'y_centroid']].T.to_dict('list') # positions of nodes to plot

In [ ]:
# draw networkx graph
# first draw edges
ax=nx.draw(G,
        positions, # use centroids for positions
        width = 0.1,
        node_size=0) # show thin edges
# then draw base nodes
nx.draw_networkx_nodes(G,
        positions, # use centroids for positions
        node_size = 100,
        nodelist = list(bases), # only showing bases 
        ax=ax)
plt.savefig('base_locations.svg', bbox_inches='tight')

In [ ]:
# save angle from first base to cell
game_grid['THETA'] = np.arctan2(game_grid['y_centroid']-game_grid.loc[bases[0],'y_centroid'],
                                game_grid['x_centroid']-game_grid.loc[bases[0],'x_centroid'])
game_grid.plot('THETA',figsize=(10,10))
plt.savefig('cell_angles.svg', bbox_inches='tight')

In [ ]:
# save distance from first base to cell
game_grid['DISTANCE'] = pd.Series(nx.single_source_shortest_path_length(G,bases[0]))

In [ ]:
location_type='all'

In [ ]:
# load locations location from file and convert to same CRS as game_grid
shelters=gpd.read_file(f'facilities/shelters.shp').to_crs('epsg:4269')
hospitals=gpd.read_file(f'facilities/hospitals.shp').to_crs('epsg:4269')
powerplants=gpd.read_file(f'facilities/powerplants.shp').to_crs('epsg:4269')
celltowers=gpd.read_file(f'facilities/cell_towers.shp').to_crs('epsg:4269')
ems=gpd.read_file(f'facilities/EMS.shp').to_crs('epsg:4269')

In [ ]:
locations=shelters.append(hospitals).append(powerplants).append(celltowers).append(ems)

In [ ]:
len(locations)

In [ ]:
# convert locations to cell locations
locations_gdf=gpd.sjoin(game_grid,locations,how='inner',op='contains')

In [ ]:
ax=game_grid.plot('Transport',figsize=(20,20),cmap='copper')
locations.plot(figsize=(20,20),ax=ax)
plt.savefig(f'locations_{location_type}.svg', bbox_inches='tight')

In [ ]:
# time for every cell is 20 minutes
game_grid['TIME'] = 20
locations_gdf['TIME'] = 20

In [ ]:
locations_gdf # print merged GeoDataFrame with cells that contain locations

In [ ]:
# divide the cells around the first base into 38 different regions based upon angle
# all regions should take the same amount of time
n=100
sorted_locations=locations_gdf.sort_values('THETA')
total=0
region=0
splits=[]
region_splits=np.linspace(0,locations_gdf['TIME'].sum(),n-5)
for i,x in sorted_locations.iterrows():
    if i in locations_gdf.index and not np.isnan(x['TIME']):
        total+=x['TIME']
        if total>=region_splits[region+1]:
            splits.append(i)
            region+=1

In [ ]:
# sort game_grid by angle
game_grid = game_grid.sort_values('THETA')

In [ ]:
# set region for each sector based upon start and stop cell names
for i,a,b in zip(range(len(splits)),splits,np.roll(splits,1)):
    game_grid.loc[b:a,'REGION']=i

In [ ]:
# fix error where first region is set to np.nan
game_grid.loc[game_grid['REGION'].isna(),'REGION'] = 0

In [ ]:
# change region for cape into two seperate regions so trucks don't travel long distances
ax=game_grid.plot('REGION',figsize=(10,10))
cape=Polygon([[-70.9,41.4],[-69.7,41.4],[-70.1,42.4]])
cape_small1=Polygon([[-70.9,41.4],[-69.5,41.4],[-70.2,42.0]])
cape_small2=Polygon([[-70.9,41.4],[-70.0,41.4],[-70.2,42.0]])
cape_small3=Polygon([[-70.9,41.4],[-70.15,41.4],[-70.42,42.0]])
cape_small4=Polygon([[-70.9,41.4],[-70.5,41.4],[-70.42,42.0]])
cape_small5=Polygon([[-70.9,41.4],[-70.7,41.4],[-70.42,42.0]])
ax=plt.plot(*cape.exterior.xy)
ax=plt.plot(*cape_small1.exterior.xy)
ax=plt.plot(*cape_small2.exterior.xy)
ax=plt.plot(*cape_small3.exterior.xy)
ax=plt.plot(*cape_small4.exterior.xy)
ax=plt.plot(*cape_small5.exterior.xy)

In [ ]:
# set region for each cape 
game_grid.loc[game_grid.within(cape),'REGION'] = n-1
game_grid.loc[game_grid.within(cape_small1),'REGION'] = n-2
game_grid.loc[game_grid.within(cape_small2),'REGION'] = n-3
game_grid.loc[game_grid.within(cape_small3),'REGION'] = n-4
game_grid.loc[game_grid.within(cape_small4),'REGION'] = n-5
game_grid.loc[game_grid.within(cape_small5),'REGION'] = n-6

In [ ]:
# combine grid cells into polygon for each region
region_polygons=gpd.GeoDataFrame(geometry=game_grid.groupby('REGION').apply(lambda x:x.unary_union))

In [ ]:
# choose colors for 3-coloring
colors=np.arange(len(region_polygons))%3

In [ ]:
# plot the regions
region_polygons.plot(colors,figsize=(10,10),cmap='Set3')
plt.savefig(f'regions_{location_type}.svg', bbox_inches='tight')

In [ ]:
region_polygons.loc[95.0].geometry

In [ ]:
# set regions in locations GeoDataFrame to be same as in game_grid
locations_gdf['REGION'] =game_grid['REGION']

In [ ]:
locations_gdf.groupby('REGION').count()

In [ ]:
# get node ids for every region
region_ids = locations_gdf.groupby('REGION').apply(lambda x: list(x.index)).to_list()

In [ ]:
# add first base to all regions, so it is added in the route
for x in region_ids:
    for base in bases:
        if base not in x:
            x.append(base)

In [ ]:
# list of all nodes possible to visit
graph_nodes=list(locations_gdf.index)+list(bases)

In [ ]:
# calculate completed subgraph on nodeList
def shortestPathNetwork(nodeList, ogNetwork):
    shortest_path_matrix = np.zeros([len(nodeList),len(nodeList)])
    for idx_i, orig in enumerate(tqdm(nodeList)):
        shortest_paths = nx.single_source_dijkstra_path_length(ogNetwork, orig, weight='weight')
        for idx_j, dest in enumerate(nodeList):
            shortest_path_matrix[idx_i, idx_j] = shortest_paths[dest]
    smallGraph = nx.from_numpy_matrix(shortest_path_matrix, create_using=nx.MultiDiGraph)
    smallGraph = nx.relabel_nodes(smallGraph,{k:v for k, v in zip(smallGraph.nodes, nodeList)})
    return smallGraph

In [ ]:
# calculate complete graph for all shelers
graph=shortestPathNetwork(graph_nodes,G)

In [ ]:
# calculate complete graph for every region where all nodes are connected
graphs = [graph.subgraph(x) for x in region_ids]

In [ ]:
get_length_of_path = lambda path, temp_graph: np.sum([temp_graph.get_edge_data(i, j, 0)['weight'] for i,j in zip(path,np.roll(path,1)) if i!=j])

In [ ]:
# for every graph, calculate the TSP (Travelling Salesman Problem) using greedy and refine with 2-opt
# print the time to travel over every path
import itertools
def calculate_paths(graphs,base):
    paths=[]
    for g in graphs:
        temp_graph=g.copy()
        temp_graph.remove_node(bases[1-bases.index(base)])
        node=base
        path=[]
        while len(temp_graph)>1:
            path.append(node)
            arr={x:temp_graph[node][x][0]['weight'] for x in temp_graph.neighbors(node)}
            temp_graph.remove_node(node)
            node=min(arr,key=arr.get)
        temp_graph=g.copy()
        temp_graph.remove_node(bases[1-bases.index(base)])
        length=get_length_of_path(path,temp_graph)
        best_distance,new_distance=np.inf,0
        while new_distance<best_distance:
            for i, k in itertools.combinations(range(len(path)),r=2):
                new_route = path[:i]+list(reversed(path[i:k+1]))+path[k+1:]
                new_distance = get_length_of_path(new_route,temp_graph)
                if new_distance<best_distance:
                    path,best_distance = new_route, new_distance
        paths.append((path,(get_length_of_path(path,temp_graph)*120/100+len(temp_graph)*20)/60))
    return paths

In [ ]:
# calculate minimum paths and lengths
paths, lengths=list(zip(*[x if x[1]<=y[1] else y for x,y in zip(*[calculate_paths(graphs,base) for base in bases])]))

In [ ]:
lengths

In [ ]:
new_path=[]
new_path2=[]

In [ ]:
for x in paths[n-6]:
    if game_grid.loc[x,'y_centroid']<=41.71592941653868:
        new_path.append(x)
    else:
        new_path2.append(x)

In [ ]:
new_paths=[]

In [ ]:
for i in range(n):
    if i==n-6:
        new_paths.append([bases[1]]+new_path)
    else:
        new_paths.append(paths[i])
new_paths.append(new_path2)

In [ ]:
graphs.append(graphs[n-6].copy())

In [ ]:
new_lengths=[]

In [ ]:
for i in range(n):
    if i==n-6:
        new_lengths.append((get_length_of_path(new_paths[n-6],graphs[n-6])*120/100+len(new_paths[n-6])*20)/60)
    else:
        new_lengths.append(lengths[i])
new_lengths.append((get_length_of_path(new_paths[n],graphs[n-6])*120/100+len(new_paths[n])*20)/60)

In [ ]:
# graph all of the paths
pos=game_grid[['x_centroid','y_centroid']].T.to_dict('list')
edge_data=[]
for i, temp_graph,path in list(zip(range(n+1),graphs,new_paths)):
    weights=nx.get_edge_attributes(temp_graph,'weight')
    edge_data+=[ (x,i%12) for x in zip(path,np.roll(path,1)) if x[0]!=x[1]]
edges,weights=zip(*edge_data)
ax=game_grid.plot('Transport',figsize=(20,20),alpha=0.5,cmap='copper')
nx.draw(G,pos=pos,edgelist=edges,node_size=0,width=3,edge_color=weights,ax=ax,edge_cmap=plt.cm.Set3)
plt.savefig(f'paths_{location_type}.svg', bbox_inches='tight')

In [ ]:
final_paths=[np.roll(np.array(p),-np.where(np.array(p)==bases[0])[0]) for p in new_paths if bases[0] in np.array(p)]
final_paths+=[np.roll(np.array(p),-np.where(np.array(p)==bases[1])[0]) for p in new_paths if bases[1] in np.array(p)]

In [ ]:
import csv
with open('paths_all_locations.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['LENGTH','PATH'])
    writer.writerows(zip(new_lengths,final_paths))

In [ ]:
expanded = []
for pa in final_paths:
    pa=list(pa)
    pa.append(pa[0])
    newPa = [pa[0]]
    for i in range(len(pa)-1):
        newPa += nx.shortest_path(G, source=pa[i], target=pa[i+1], weight='weight')[1:]
    expanded.append(newPa)

In [ ]:
colorList = ['#8dd3c7','#ffffb3','#bebada','#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#bc80bd','#ccebc5','#ffed6f'] * 20

ax = game_grid.plot('Transport', figsize=(20,20), cmap='binary', alpha=0.4)
pos = game_grid[['x_centroid', 'y_centroid']].T.to_dict('list')
for idx, path in tqdm(enumerate(expanded),total=n+1):
    edges = [[i,j] for i,j in zip(path, np.roll(path,1)) if  i!=j]
    nx.draw(G,pos=pos,edgelist=edges,node_size=0,width=3,edge_color=colorList[idx],ax=ax)
# nx.draw_networkx_nodes(G,pos=pos, nodelist=list(joinedLocs.copy().reset_index()['MGRS']),node_color='r',node_size=3)
plt.savefig(f'paths_finegrain_{location_type}.png', dpi=100)